In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import argparse
import string
import json
import csv

In [ ]:
consumer_key = 'oKebnsv4ofhF1AwmEfkV9CGbG'
consumer_secret = 'hICgy8pgfx7omYz0vRExanOrl0ejOfJkPVDMVrNx6ZWb2vkCWw'
access_token = '2974494974-9nu3LxdZ3CTNtvVcZnPeKNPXrst7wHEoVRm5Xu1'
access_secret = 'CF8FLLntfOwNPTBkAgX0TiwHwWs52NVIiny12ZOoyY3k2'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [ ]:
all_expert_tweets = []
def get_all_tweets(screen_name):
    with open('C:\\Users\\100631155\\Recruiter\\%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        for tweet in tweepy.Cursor(api.user_timeline,id=screen_name).items():
            userid = tweet.id_str
            text=tweet.text.encode("utf-8")
            all_expert_tweets.append(text)
            time = tweet.created_at
            writer.writerow([userid,time,text])

# Perform a user search 

In [ ]:
results = api.search('job recruiter')
for user in results:
    print(user.id, user.author.screen_name)
    get_all_tweets(user.author.screen_name)

In [ ]:
verbose = 1

In [ ]:
from __future__ import print_function
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

import numpy as np
import twitter

# Getting the Tweets

In [ ]:
def get_tweets():
    tweets = []
    max_id = None
    for _ in range(100):
        tweets.extend(list(api.GetUserTimeline(screen_name='Hire_Colorado',
                                               max_id=max_id,
                                               count=200,
                                               include_rts=False,
                                               exclude_replies=True)))
        max_id = tweets[-1].id
    return [tweet.text for tweet in tweets]

np.save(file='meni_tweets.npy', arr=get_tweets())

In [ ]:
CORPUS_LENGTH = None

def get_corpus(verbose=0):
    tweets = np.load('meni_tweets.npy')
    tweets = [t for t in tweets if 'http' not in t]
    tweets = [t for t in tweets if '&gt' not in t]
    corpus = u' '.join(tweets)
    global CORPUS_LENGTH
    CORPUS_LENGTH = len(corpus)
    if verbose:
        print('Corpus Length:', CORPUS_LENGTH)
    return corpus

corpus = get_corpus(verbose=verbose)

In [ ]:
import sys
!{sys.executable} -m pip install -U gensim

# Language modeling

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re
import csv
import random
from sklearn.preprocessing import LabelEncoder
from gensim.models import word2vec
from sklearn.utils import shuffle
from nltk import word_tokenize

# Convert label to numeric vector

In [ ]:
def label_convert(data):
    enc = LabelEncoder()
    nLabels = len(set(enc.fit(data).classes_))
    data = enc.transform(data)
    return np.eye(nLabels)[data].astype("int32")

# Process tweets

In [ ]:
def process_line(line):    
    if re.search("http[\S]+", line):
        line = re.sub("http[\S]+","<url>", line)        
    line = re.sub("\Wamp+","and", line)    
    if re.search("[A-Za-z]\W\w+",line):
        line  = re.sub("[^A-Za-z\d\s\'\˘\’\.\,]+"," ", line)
    line = line.lower()   
    cleanLine = word_tokenize(line)
    return cleanLine

# Single string for word to vec

In [ ]:
def convert_single_string(dataset):
    q = " "
    stringData = []
    for row in dataset:
        line = q.join(row)
        stringData.append(line)
    string = q.join(stringData) 
    return {"single_string" : string, "lines" : stringData }

# Plotter

In [ ]:
def plot(x1, y1, x2, y2, label1 = None, label2 = None, title = None, x_label = None, y_label = None):
    fig = plt.figure(figsize=(8,8))
    ax1 = fig.add_subplot(1,1,1)
    ax1.plot(x1, y1, color='navy', label = label1)
    ax1.tick_params(bottom='off',top='off',left='off',right='off')
    ax1.spines["right"].set_visible(False)
    ax1.spines["left"].set_visible(False)
    ax1.spines["top"].set_visible(False)
    ax1.spines["bottom"].set_visible(False)
    ax1.set(title= title, ylabel= y_label, xlabel = x_label)
    ax1.plot(x2, y2,color= 'green', label = label2)
    ax1.legend(loc= 'upper center', bbox_to_anchor=(0.5, -0.1),  shadow=True, ncol=2)
    ax1.grid()
    plt.savefig("{}.png".format(title))
    plt.show()

# Read files

In [ ]:
f = open('train.csv','r', encoding = 'utf8')
trainData = list(csv.reader(f))[1:]
f = open('test.csv','r', encoding = 'utf8')
testData = list(csv.reader(f)) [1:]

# Shuffle training data 

In [ ]:
random.shuffle(trainData)

# Get target and raw input

In [ ]:
trainTarget, trainSentences = zip(*trainData) #trainIds
#testIds, testTarget, testSentences = zip(*testData) 

noRows = len(trainSentences)
trainTarget = label_convert(trainTarget)
#testTarget = label_convert(testTarget)

# Clean data

In [ ]:
cleanTrain = [process_line(row) for row in trainSentences]
#cleanTest = [process_line(row) for row in testSentences]

whole = cleanTrain + cleanTest

# Train word to vec

In [ ]:
model = word2vec.Word2Vec(whole, size  = 30, window = 10, min_count=1, iter = 20)

In [ ]:
maxLength = 35 #max(trainLen)

trainLen = [min(len(each), maxLength) for each in cleanTrain]
testLen = [min(len(each), maxLength) for each in cleanTest]

# Train Data

In [ ]:
row_count = 0
trainNumeric = np.zeros((len(cleanTrain), maxLength), dtype='int32')

for row in cleanTrain:
    index_count = 0
    for each in row:
        try:
            trainNumeric[row_count][index_count] = model.wv.vocab[each].index
        except:
            trainNumeric[row_count][index_count] = 0
        index_count += 1
        if index_count >= maxLength:
            print("Done with the Row: ", row_count)
            break       
    row_count += 1

# Test Data

In [ ]:
row_count = 0
testNumeric = np.zeros((len(cleanTest), maxLength), dtype='int32')

for row in cleanTest:
    index_count = 0
    for each in row:
        try:
            testNumeric[row_count][index_count] = model.wv.vocab[each].index
        except:
            testNumeric[row_count][index_count] = 0
        index_count += 1
        if index_count >= maxLength:
            print("Done with the Row: ", row_count)
            break       
    row_count += 1

In [ ]:
embeddingMatrix = np.zeros((len(model.wv.vocab), 30), dtype="float32")
for i in range(len(model.wv.vocab)):
    embeddingVector = model.wv[model.wv.index2word[i]]
    if embeddingVector is not None:
        embeddingMatrix[i] = embeddingVector

# Model 1 Parameters

In [ ]:
ValidIndexStart = 4199

batchSize = 100
lstmUnits = 35
numClasses = 2

epochs = 15
display_step = 1

# Define model

In [ ]:
tf.reset_default_graph()

input_data = tf.placeholder(tf.int32, [None, maxLength])

input_length = tf.placeholder(tf.int32, [None])

labels = tf.placeholder(tf.int32, [None, numClasses])

# Weight & bias

In [ ]:
W =tf.Variable(tf.random_normal([lstmUnits, numClasses]))
b = tf.Variable(tf.random_normal([numClasses]))

In [ ]:
data = tf.nn.embedding_lookup(embeddingMatrix,input_data)

lstmCell = tf.contrib.rnn.LSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.MultiRNNCell([lstmCell])
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.5)

value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32, sequence_length=input_length)

word_index = tf.reshape(input_length, [tf.shape(value)[0]])-1

sentence_index = tf.range(0, tf.shape(input_data)[0]) * (tf.shape(input_data)[1])
                      
index = sentence_index + word_index

flat = tf.reshape(value, [-1, tf.shape(value)[2]])
        
last = tf.gather(flat, index)

out_layer = tf.add(tf.matmul(last,W), b)

prediction = tf.nn.softmax(out_layer)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))

accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer,
                                                              labels=labels))

optimizer = tf.train.AdamOptimizer().minimize(loss)

init = tf.global_variables_initializer()

# Run Model

In [ ]:

valLoss1 = [] #results for validation loss
trainLoss1 = [] #train loss
valAcc1 = []
trainAcc1 = []

with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
    #train model
    ValNumeric, ValLen, ValTarget = trainNumeric[ValidIndexStart+1: ], trainLen[ValidIndexStart+1: ], trainTarget[ValidIndexStart+1: ]  
    trainNumeric_, trainLen_, trainTarget_ = trainNumeric[:ValidIndexStart], trainLen[:ValidIndexStart], trainTarget[:ValidIndexStart]
    
    for epoch in range(epochs):
        nBatch = int( trainNumeric.shape[0] / batchSize )
        trainNumeric_, trainLen_, trainTarget_ = shuffle(trainNumeric_, trainLen_, trainTarget_)
        trainBatchesSentenceSplit = np.array_split(trainNumeric_, nBatch)
        trainBatchesLenSplit = np.array_split(trainLen_, nBatch)
        trainBatchesLabelSplit = np.array_split(trainTarget_, nBatch)
        
        for i in range(nBatch):
            batch = trainBatchesSentenceSplit[i]
            length = trainBatchesLenSplit[i]
            target = trainBatchesLabelSplit[i]
        # Run optimization op (backprop) and cost op (to get loss value)
            _= sess.run([optimizer], feed_dict={input_data: batch,
                                                  input_length: length,
                                                  labels: target
                                                  })
        print("Epoch Done:{}".format(epoch))
        # Display logs per epoch
        if epoch % display_step == 0:
            trainLoss, trainAcc = sess.run([loss, accuracy], feed_dict = {input_data:trainNumeric_,
                                                                          input_length:trainLen_, 
                                                                          labels:trainTarget_         
                                                                           })
            print("-"*60)
            print("Epoch: {}\nTrain Batch:\nLoss:{:0.4f}\n Accuracy: {:0.3f}\n".format(epoch,trainLoss,trainAcc))
            
            # Validate model
            ValLoss, ValAcc = sess.run([loss, accuracy],feed_dict = {input_data: ValNumeric,
                                                                     input_length: ValLen,                               
                                                                     labels: ValTarget 
                                                                    })
            print("Validation Batch:\n Loss:{:0.3f}\n Accuracy:{:0.3f}".format(ValLoss,ValAcc))
            
            trainLoss1.append([epoch,trainLoss])
            trainAcc1.append([epoch,trainAcc])
            valLoss1.append([epoch,ValLoss])
            valAcc1.append([epoch,ValAcc])                
                
    print("Optimization Finished!")
    # Test model
    testPrediction, testAcc, testLoss = sess.run([prediction, accuracy, loss], feed_dict = {input_data: testNumeric, input_length: testLen, labels: testTarget} )

# Formatting for plots

In [ ]:
x1, y1 = zip(*valLoss1)
x2, y2 = zip(*trainLoss1)
x3, y3 = zip(*valAcc1)
x4, y4 = zip(*trainAcc1)

In [ ]:
f = plt.figure()
plt.plot(x1, y1, x2, y2, label1 = "Validation Loss", label2 = "Training Loss", title = "Model 1 Loss",y_label = "Loss", x_label = "Epoch")
f.savefig("Loss", bbox_inches='tight')
plot(x3, y3, x4, y4, label1 = "Validation Accuracy", label2 = "Training Accuracy", title = "Model 1 Accuracy",y_label = "Accuracy", x_label = "Epoch") 

# Get predictions for test data

In [ ]:
name = "fp.1"
predictions_test_1 = [row for row in testPrediction]
        
predictions_test_1_id = []        
for i, row in enumerate(predictions_test_1):
    row = row.tolist()
    row.insert(0,i)
    predictions_test_1_id.append(row)
predictions_test_1_id.insert(0, ["id", "experts", "notExperts"])

with open(name + ".csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(predictions_test_1_id)

In [ ]:
import networkx as net
import matplotlib.pyplot as plt

from collections import defaultdict
import math

twitter_network = [ line.strip().split('\t') for line in file('twitter_network.csv') ]

o = net.DiGraph()
hfollowers = defaultdict(lambda: 0)
for (twitter_user, followed_by, followers) in twitter_network:
    o.add_edge(twitter_user, followed_by, followers=int(followers))
    hfollowers[twitter_user] = int(followers)

SEED = 'jasonrmartin'

# centre around the SEED node and set radius of graph
g = net.DiGraph(net.ego_graph(o, SEED, radius=4))

def trim_degrees_ted(g, degree=1, ted_degree=1):
    g2 = g.copy()
    d = net.degree(g2)
    for n in g2.nodes():
        if n == SEED: continue # don't prune the SEED node
        if d[n] <= degree and not n.lower().startswith('ted'):
            g2.remove_node(n)
        elif n.lower().startswith('ted') and d[n] <= ted_degree:
            g2.remove_node(n)
    return g2

def trim_edges_ted(g, weight=1, ted_weight=10):
    g2 = net.DiGraph()
    for f, to, edata in g.edges_iter(data=True):
        if f == SEED or to == SEED: # keep edges that link to the SEED node
            g2.add_edge(f, to, edata)
        elif f.lower().startswith('ted') or to.lower().startswith('ted'):
            if edata['followers'] >= ted_weight:
                g2.add_edge(f, to, edata)
        elif edata['followers'] >= weight:
            g2.add_edge(f, to, edata)
    return g2

print(len(g))
core = trim_degrees_ted(g, degree=235, ted_degree=1)
print('core after node pruning: ', len(core))
core = trim_edges_ted(core, weight=250000, ted_weight=35000)
print('core after edge pruning: ', len(core))

nodeset_types = { 'TED': lambda s: s.lower().startswith('ted'), 'Not TED': lambda s: not s.lower().startswith('ted') }

nodesets = defaultdict(list)

for nodeset_typename, nodeset_test in nodeset_types.iteritems():
    nodesets[nodeset_typename] = [ n for n in core.nodes_iter() if nodeset_test(n) ]

pos = net.spring_layout(core) # compute layout

colours = ['red','green']
colourmap = {}

plt.figure(figsize=(18,18))
plt.axis('off')

# draw nodes
i = 0
alphas = {'TED': 0.6, 'Not TED': 0.4}
for k in nodesets.keys():
    ns = [ math.log10(hfollowers[n]+1) * 80 for n in nodesets[k] ]
    print(k, len(ns))
    net.draw_networkx_nodes(core, pos, nodelist=nodesets[k], node_size=ns, node_color=colours[i], alpha=alphas[k])
    colourmap[k] = colours[i]
    i += 1
print('colourmap: ', colourmap)

# draw edges
net.draw_networkx_edges(core, pos, width=0.5, alpha=0.5)

# draw labels
alphas = { 'TED': 1.0, 'Not TED': 0.5}
for k in nodesets.keys():
    for n in nodesets[k]:
        x, y = pos[n]
        plt.text(x, y+0.02, s=n, alpha=alphas[k], horizontalalignment='center', fontsize=9)
